# Datasets Test

Test the various kind of data sets that come with TensorTools.

*Remarks: The order of the image outputs can change between single executions!*

In [ ]:
# Force matplotlib to use inline rendering
%matplotlib inline

import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import numpy as np
import tensorflow as tf
import tensortools as tt

## MNIST

In [ ]:
mnist_train = tt.datasets.mnist.MNISTTrainDataset()
mnist_valid = tt.datasets.mnist.MNISTValidDataset()
mnist_test = tt.datasets.mnist.MNISTTestDataset()

In [ ]:
def display_mnist(dataset):
    x, y = dataset.get_batch(1)
    tt.visualization.display_array(x[0] * 255)
    print('Label: {}'.format(y))
    
display_mnist(mnist_train)
display_mnist(mnist_valid)
display_mnist(mnist_test)

## MovingMNIST

In [ ]:
moving_train = tt.datasets.moving_mnist.MovingMNISTTrainDataset(input_seq_length=5, target_seq_length=5)
moving_valid = tt.datasets.moving_mnist.MovingMNISTValidDataset(input_seq_length=5, target_seq_length=5)
moving_test = tt.datasets.moving_mnist.MovingMNISTTestDataset(input_seq_length=5, target_seq_length=5)

In [ ]:
def display_moving(dataset, title):
    x, y = dataset.get_batch(1)
    full_seq = np.concatenate((x[0], y[0]))
    tt.visualization.display_batch(full_seq, nrows=2, ncols=5, title=title)
    
display_moving(moving_train, 'Train')
display_moving(moving_valid, 'Validation')
display_moving(moving_test, 'Test')

## UCF11

*Remarks: This example uses an input queue.*

In [ ]:
ucf_train = tt.datasets.ucf11.UCF11TrainDataset(input_seq_length=3, target_seq_length=3,
                                                image_size=(60, 80, 3))

In [ ]:
batch_x, batch_y = ucf_train.get_batch(32)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    try:
        print('Starting queue runners...')
        x, y = sess.run([batch_x, batch_y])       
    except tf.errors.OutOfRangeError:
        print("Done training -- epoch limit reached")
    finally:
        # When done, ask the threads to stop
        coord.request_stop()

    coord.join(threads)
    
def display_ucf(x, y, title):
    x = (x + 1) * 127.5
    y = (y + 1) * 127.5
    print("x-range: [{}, {}], y-range: [{}, {}]".format(x.min(), x.max(), y.min(), y.max()))
    full_seq = np.concatenate((x[0], y[0]))
    tt.visualization.display_batch(full_seq, nrows=2, ncols=3, title=title)

display_ucf(x, y, 'Train')